In [1]:
!pip install transformers seqeval shap 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=6ac0cbee37ab15a73df587e3db8eccfe6fa329c52d132c42c9b5cadbb0267d38
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
import pandas as pd
import numpy as np
import torch
import transformers
import sys
import seqeval
import os
import spacy
import shap
import scipy as sp

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 加载模型

In [4]:
remote = True
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 加载模型
# 加载预训练模型
model_name = 'bert-base-uncased' # 预训练模型名字
tokenizer = transformers.BertTokenizer.from_pretrained(model_name)
model = transformers.BertForSequenceClassification.from_pretrained(model_name, num_labels=2) # 需要预测的类别数为2
optimizer = transformers.AdamW(model.parameters(), lr=2e-5, eps=1e-8)
if remote:
  os.chdir('/content/drive/MyDrive/ModelDebug')
  checkpoint = torch.load('classifier/model.pt',map_location=torch.device(device))
  # df = pd.read_csv('classifier/textNFR.csv')
  df = pd.read_csv('classifier/new_dataset2.csv')
  #os.chdir('examples')
  from examples.infer_softmax_ner import predict
else:
  checkpoint = torch.load('model.pt',map_location=torch.device(device))
  os.chdir('../classifier')
  df = pd.read_csv('textNFR.csv')
  from examples.infer_softmax_ner import predict
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()
model.to(device)

cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

/content/drive/MyDrive/ModelDebug


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# 执行模型优化

## 可解释性分析

In [5]:
def f(text):
    #输入文本，返回预测值
    tv = torch.tensor([tokenizer.encode(t, padding='max_length', max_length=128, truncation=True) for t in text]).to(device)
    attention_mask = (tv!=0).type(torch.int64).to(device)
    outputs = model(tv,attention_mask=attention_mask)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores)
    return val

In [6]:
# 使用 shap 进行可解释性分析
explainer = shap.Explainer(f,tokenizer,output_names=["FR","NFR"])

In [7]:
#text列表转json列表
def toJson(text):
    return {'sent':text}
def getTokens(doc):
    nlp = spacy.load('en_core_web_sm')
    tokens= []
    doc = nlp(doc)
    for t in doc:
        tokens.append(t.text)
    return tokens
#抽取每个样例的关注点
#输入data是一个dict，dict中有一个key为sent，value为句子
def getConcerns(data):
    res = []
    tokens = getTokens(data['sent'])
    cons = predict([data])[0]
    if len(tokens)!=len(cons):
        print("tokens和concerns长度不一致"+str(len(tokens))+" "+str(len(cons)))
    for t,c in zip(tokens,cons):
        if c!='O':
            res.append(t)
    return res
#计算重要特征，返回特征与重要性
def important(shap_values):
    reason = []
    for ele in shap_values:
        sum =0
        for num in ele.values[0]:
            sum += abs(num[0]) + abs(num[1])  #两个分类的重要性之和
        avg = sum/(len(ele.values[0])*2)
        res = []
        for values,datas in zip(ele.values[0],ele.data[0]):
            cur = []
            if abs(values[0])+abs(values[1])>=avg*2:
                res.append(datas)
        reason.append(res)
    return reason
#检测噪声(重要但是不是关注点)
def errCheck(importance,concerns):
    errList = []
    for imp,con in zip(importance,concerns):
        imp=imp.strip()
        if(imp in concerns): continue
        else : errList.append(imp)
    return errList

In [21]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# 初始化停用词列表
def init_stopwords(language='english'):
    return set(stopwords.words(language))

# 判断噪声是否是停用词
def is_noise_stopword(noise_word, stopwords_set):
    return noise_word.lower() in stopwords_set

# 统计噪声中停用词的数量
def count_noise_stopwords(noise_words, stopwords_set):
    count = 0
    for word in noise_words:
        if is_noise_stopword(word, stopwords_set):
            count += 1
    return count

# 创建新数据集，判断噪声是否是停用词并统计数据
def createNewDataset(dataset, label, stopwords_set):
    new_dataset = []
    index = 0
    total_words = 0
    total_noise_stopwords = 0

    for d, l in zip(dataset, label):
        try:
            ls = []
            val = explainer([d])  # 计算每个样例的shap值
            imp = important([val])  # 计算每个样例的重要单词
            cons = getConcerns(toJson(d))  # 计算每个样例的关注点单词
            errlist = errCheck(imp[0], cons)  # 计算每个样例的噪声

            noise_stopwords_count = count_noise_stopwords(errlist, stopwords_set)
            total_noise_stopwords += noise_stopwords_count

            total_words += len(errlist)

            for e in errlist:
                d = d.replace(e, '')

            ls.append(index)
            ls.append(d)
            ls.append(str(l))
            new_dataset.append(ls)
            index += 1
        except:
            print("error in sent:" + d)
            continue

    noise_stopwords_ratio = total_noise_stopwords / total_words
    return new_dataset, noise_stopwords_ratio



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
d = dataset[2]
stopwords_set = init_stopwords(language='english')
val = explainer([d])  # 计算每个样例的shap值
imp = important([val])  # 计算每个样例的重要单词
cons = getConcerns(toJson(d))  # 计算每个样例的关注点单词
errlist = errCheck(imp[0], cons)  # 计算每个样例的噪声
count = count_noise_stopwords(errlist, stopwords_set)
print(count)
print(errlist)

Evaluating: 100%|██████████| 1/1 [00:00<00:00, 26.55it/s]

tokens和concerns长度不一致26 20
1
['if', '90', 'event', 'activity']


In [22]:
dataset = df['RequirementText']
label = df['NFR']
new_dataset,ratio = createNewDataset(dataset,label,init_stopwords(language='english'))
print(ratio)

Evaluating: 100%|██████████| 1/1 [00:00<00:00, 39.56it/s]


tokens和concerns长度不一致9 8


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 56.34it/s]


tokens和concerns长度不一致15 13


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 50.44it/s]


...

tokens和concerns长度不一致23 22


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 48.00it/s]


tokens和concerns长度不一致29 31


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 40.97it/s]

tokens和concerns长度不一致46 40
0.26031746031746034
